# 1. A database for a time-domain survey (Part I): creation of a database

In [15]:
##     Include all required python-related packages     ##
import sqlite3 as sq #handle sql databases
from astropy.table import vstack, Table #read tables
import pandas as pd #dataframes
import os #system 
import numpy as np #deal with arrays
#import matplotlib.pyplot as plt #plotting
#to show plots in line
%matplotlib inline 

###Reading and handeling initial tables
I use Astropy to handle given tables with format .csv and .fits. The directory needs to be changed in case this ipython is re-run in another location.

There are two types of tables: an information/summary table (Info), and tables containing data (ended in .fits). In total we have 19 tables (1 Info and 18 data tables).

The **aim** is to create only 3 tables: an updated Info table, one big table containing information of the rest of 18 tables, and a color data. This will be the started point of the data base to create. For further information about the reasons why only 3 tables will be considered for the data base, the reader is kindly referred to the main report in the github repository.

In [16]:
directory = '/Users/guadalupecanasherrera/DDM2017_FinalProject/FinalProject/Q1/Tables/'
Info=Table.read(directory+'file_info_for_problem.csv',format='ascii.csv')
#Info #comment if you do not want to show this table

In [17]:
# Counting, saving and listing data tables in the directory
items = os.listdir(directory)
data_tables_fits = []
for names in items:
    if names.endswith(".fits"):
        data_tables_fits.append(names)

print "Number of data tables: {}".format(len(data_tables_fits))

#data_tables_fits #Un/Comment if you do/not want to see the name of the files

Number of data tables: 18


In [18]:
#The names of the .fits in reality and in the Info table does not agree. Therefore, I add a column in info with the
#real file name in the order that correspond taking into consideration the date.

data_tables = ['Field-1-Z','Field-1-J','Field-1-H','Field-1-Ks-E002','Field-1-Ks-E001',
            'Field-1-Ks-E003','Field-1-Y','Field-2-Z','Field-2-J','Field-2-H',
            'Field-2-Ks-E001','Field-2-Y','Field-3-Z','Field-3-J','Field-3-H',
             'Field-3-Ks-E002','Field-3-Ks-E001','Field-3-Y']

data_tables_fits=[]
for files in data_tables:
    data_tables_fits.append(str(files)+'.fits')

In [19]:
# Adding another column to Info to include the real name of the table files instead of the automatic .fits name
Info['RealFilename']=data_tables_fits
Info #Un/Comment if you do/not want to see the name of the files

ID,FieldID,Filename,Filter,MJD,Airmass,Exptime,RealFilename
int64,int64,str36,str2,float64,float64,float64,str20
1,1,Z-ADP.2017-01-18T11:58:36.905.fits,Z,57267.1671072,1.6405,40.0,Field-1-Z.fits
2,1,J-ADP.2017-01-18T11:58:35.781.fits,J,57257.0504323,1.0105,48.0,Field-1-J.fits
3,1,H-ADP.2017-01-18T11:58:35.780.fits,H,57257.044108,1.006,16.0,Field-1-H.fits
4,1,Ks-ADP.2016-05-25T15:33:39.546.fits,Ks,56788.346937,1.023,16.0,Field-1-Ks-E002.fits
5,1,Ks-ADP.2017-01-18T11:58:39.907.fits,Ks,56561.0020158,1.079,16.0,Field-1-Ks-E001.fits
6,1,Ks-ADP.2016-05-25T15:33:43.377.fits,Ks,56829.0390512,1.6085,16.0,Field-1-Ks-E003.fits
7,1,Y-ADP.2017-01-18T11:58:36.901.fits,Y,57267.1596647,1.5605,40.0,Field-1-Y.fits
8,2,Z-ADP.2017-01-18T11:58:36.905b.fits,Z,57268.1671072,1.6405,40.0,Field-2-Z.fits
9,2,J-ADP.2017-01-18T11:58:35.781b.fits,J,57258.0504323,1.0105,48.0,Field-2-J.fits


In [20]:
# Reading the 18 data tables and creating only one big data table, named Data. The common variables between the Info
#table and the Data table will be ID, Filter and FieldID

Data=Table()
for ii,files in enumerate(data_tables):
#     print(files[:-5])
    Files=Table.read(directory+str(files)+'.fits',format='fits')
    #print len(Files)
    for jj, kola in enumerate(Info):
        if kola['RealFilename']==str(files)+'.fits':
            Files['ID']=np.ones(len(Files))*kola['ID']
            Files['Filter']=kola['Filter']
            Files['FieldID']=kola['FieldID']
            
    Data=vstack([Data,Files])

Data #Un/Comment if you do/not want to see the name of the files

RunningID,X,Y,Flux1,dFlux1,Flux2,dFlux2,Flux3,dFlux3,Ra,Dec,Class,Mag1,dMag1,Mag2,dMag2,Mag3,dMag3,StarID,ID,Filter,FieldID
Number,Pixels,Pixels,ADU,ADU,ADU,ADU,ADU,ADU,RADIANS,RADIANS,Flag,ADU,ADU,ADU,ADU,ADU,ADU,,,,
float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float64,str2,int64
9.0,6676.62,5.03455,2886.02,145.136,4697.14,205.506,6884.06,291.283,4.70662,-0.529796,-1.0,17.433,0.054601,17.4321,0.0475026,17.4231,0.0459405,0,1.0,Z,1
13.0,6841.46,5.31207,2585.4,144.623,4467.49,205.039,6966.38,290.953,4.7069,-0.529931,1.0,17.5525,0.0607343,17.4866,0.049831,17.4102,0.0453463,1,1.0,Z,1
15.0,6355.06,5.88212,2765.64,145.052,4436.22,205.378,6540.06,291.163,4.70609,-0.529531,-1.0,17.4793,0.0569448,17.4942,0.050265,17.4788,0.048337,2,1.0,Z,1
19.0,5897.49,7.52958,899.127,138.445,1464.07,197.921,2023.29,284.087,4.70533,-0.529153,-1.0,18.6992,0.167179,18.6978,0.146776,18.7526,0.152447,3,1.0,Z,1
22.0,5809.89,6.13442,3412.75,140.556,5444.32,200.291,8012.71,286.57,4.70518,-0.529083,-1.0,17.251,0.0447169,17.2719,0.0399432,17.2583,0.0388309,4,1.0,Z,1
14.0,6874.91,5.43629,4129.01,145.694,6773.71,206.173,9926.46,291.985,4.70695,-0.529958,-1.0,17.0442,0.0383108,17.0347,0.0330469,17.0257,0.0319368,5,1.0,Z,1
12.0,6571.8,5.21043,3762.78,145.753,6288.57,206.302,9654.43,292.267,4.70645,-0.52971,1.0,17.145,0.0420567,17.1153,0.0356186,17.0559,0.0328684,6,1.0,Z,1
17.0,6580.03,6.52743,2046.26,144.553,3359.13,204.849,4906.73,290.597,4.70647,-0.529715,1.0,17.8064,0.0766994,17.7962,0.0662116,17.7907,0.0643021,7,1.0,Z,1


###Creating table for color
I create a panda dataframe with colors J-H and Y-J as they will be required latter for further purposes: simplification of an SQL query and the simulation os stars. I use pandas for simplification in filtering as well as for the speed in masking.

In [21]:
# Transforming astropy Data table to panda dataframes and calculating colors and uncertainties.
Data_df=Data.to_pandas()
print Data_df.shape
DataH=Data_df[Data_df["Filter"]=="H"]
DataY=Data_df[Data_df["Filter"]=="Y"]
DataJ=Data_df[Data_df["Filter"]=="J"]
colorJH=DataJ.Mag1[DataJ.StarID==DataH.StarID].values-DataH.Mag1[DataH.StarID==DataJ.StarID].values
dcolorJH=np.abs(colorJH)*np.sqrt((DataJ.dMag1[DataJ.StarID==DataH.StarID].values/DataJ.Mag1[DataJ.StarID==DataH.StarID].values)**2
                        +(DataH.dMag1[DataH.StarID==DataJ.StarID].values/DataH.Mag1[DataH.StarID==DataJ.StarID].values)**2)

colorYJ=DataY.Mag1[DataY.StarID==DataJ.StarID].values-DataJ.Mag1[DataJ.StarID==DataY.StarID].values
dcolorYJ=np.abs(colorYJ)*np.sqrt((DataY.dMag1[DataY.StarID==DataJ.StarID].values/DataY.Mag1[DataY.StarID==DataJ.StarID].values)**2
                        +(DataJ.dMag1[DataJ.StarID==DataY.StarID].values/DataJ.Mag1[DataJ.StarID==DataY.StarID].values)**2)
Color_df=pd.DataFrame()
Color_df["colorJH"]=colorJH
Color_df["colorYJ"]=colorYJ
Color_df["dcolorJH"]=dcolorJH
Color_df["dcolorYJ"]=dcolorYJ

(180000, 22)


In [8]:
# I also include in this new Dataframe the StarID and the FieldID as well as the class for further classification
Color_df["StarID"]=DataY.StarID[DataY.StarID==DataJ.StarID].values
Color_df["FieldID"]=DataY.FieldID[DataY.StarID==DataJ.StarID].values
Color_df["ClassH"]=DataH.Class[DataH.StarID==DataJ.StarID].values
Color_df["ClassJ"]=DataJ.Class[DataJ.StarID==DataH.StarID].values
Color_df["ClassY"]=DataY.Class[DataY.StarID==DataJ.StarID].values
Color_df[:10]

,colorJH,colorYJ,dcolorJH,dcolorYJ,StarID,FieldID,ClassH,ClassJ,ClassY
0,2.450781,-0.697231,0.009473,0.003231,0,1,-1,-2,1
1,2.190912,-0.245317,0.006166,0.000962,1,1,-1,1,1
2,1.946388,-0.305107,0.005997,0.001234,2,1,-1,-1,1
3,1.496420,-0.102165,0.011871,0.001035,3,1,-1,-1,-1
4,1.572735,-0.264221,0.003804,0.000768,4,1,0,-1,-1
5,2.345169,-0.529789,0.005636,0.001600,5,1,-1,1,1
6,2.056660,-0.656755,0.005986,0.002228,6,1,0,1,1
7,1.848858,-0.136379,0.006354,0.000657,7,1,-1,-2,-1
8,0.996487,-1.058402,0.002680,0.002221,8,1,-1,-2,-1
9,1.708319,-0.122698,0.003523,0.000346,9,1,-1,-1,1


###Transforming the tables to a database
I use Astropy to transform the tables to panda dataframes and then, from panda dataframes to SQL using directly the package pandas to ingest the data. I design the schemas to be passed to pandas as well.

In [9]:
#Schemas: I need three different schemas for the three different tables:

Info_schema = """
                CREATE TABLE Info (ID INT, FieldID INT,  
                Filename VARCHAR(50), Filter VARCHAR(5), MJD DOUBLE, Airmass DOUBLE, Exptime DOUBLE, 
                RealFilename VARCHAR(50),
                UNIQUE(ID),
                PRIMARY KEY(ID, MJD),
                FOREIGN KEY(FieldID))
                
              """

Color_schema = """
                CREATE TABLE Color (colorJH DOUBLE, colorYJ DOUBLE, dcolorJH DOUBLE, dcolorYJ DOUBLE, 
                StarID INT,  FieldID INT,  ClassH, INT,  ClassJ INT, ClassY INT,
                UNIQUE(StarID),
                PRIMARY KEY(StarID),
                FOREIGN KEY(FieldID))
                
               """

Data_schema = """
                CREATE TABLE Data (RunningID INT, X DOUBLE, 
                Y DOUBLE, Flux1 DOUBLE, dFlux1 DOUBLE, Flux2 DOUBLE, dFlux2 DOUBLE, Flux3 DOUBLE, dFlux3 DOUBLE,
                Ra DOUBLE, Dec DOUBLE, Class INT, Mag1 DOUBLE, dMag1 DOUBLE, Mag2 DOUBLE, dMag2 DOUBLE,
                Mag3 DOUBLE, dMag3 DOUBLE, StarID INT, ID INT, Filter VARCHAR(5), FieldID DOUBLE,
                PRIMARY KEY(StarID), 
                FOREIGN KEY(FieldID, ID))
                
                """

In [10]:
# Select the name of the database: I include my student number for simplicity in case it is tested
Name_database = 'Database_s1848151.db'
# Open sql data base frame
con = sq.connect(Name_database)
# Transform Info to pandas
Info_df=Info.to_pandas()
# And create the tables into the Database
Data_df.to_sql('Data', con, schema=Data_schema, if_exists='replace')
Info_df.to_sql('Info', con, schema=Info_schema, if_exists='replace')
Color_df.to_sql('Color', con, schema=Color_schema, if_exists='replace')

### Transforming Info table to latex to be included in the report
I use the method already available in pandas to transform pandas dataframes into latex tables for a neat presentation.


In [11]:
print Info_df.to_latex(columns=["ID", "FieldID", "Filter", "MJD", "Filename", "RealFilename"])

\begin{tabular}{lrrlrll}
\toprule
{} &  ID &  FieldID & Filter &           MJD &                              Filename &          RealFilename \\
\midrule
0  &   1 &        1 &      Z &  57267.167107 &    Z-ADP.2017-01-18T11:58:36.905.fits &        Field-1-Z.fits \\
1  &   2 &        1 &      J &  57257.050432 &    J-ADP.2017-01-18T11:58:35.781.fits &        Field-1-J.fits \\
2  &   3 &        1 &      H &  57257.044108 &    H-ADP.2017-01-18T11:58:35.780.fits &        Field-1-H.fits \\
3  &   4 &        1 &     Ks &  56788.346937 &   Ks-ADP.2016-05-25T15:33:39.546.fits &  Field-1-Ks-E002.fits \\
4  &   5 &        1 &     Ks &  56561.002016 &   Ks-ADP.2017-01-18T11:58:39.907.fits &  Field-1-Ks-E001.fits \\
5  &   6 &        1 &     Ks &  56829.039051 &   Ks-ADP.2016-05-25T15:33:43.377.fits &  Field-1-Ks-E003.fits \\
6  &   7 &        1 &      Y &  57267.159665 &    Y-ADP.2017-01-18T11:58:36.901.fits &        Field-1-Y.fits \\
7  &   8 &        2 &      Z &  57268.167107 &   Z-ADP.2017-0

In [12]:
#print Data_df[:3].to_latex(columns=["RunningID", "ID", "FieldID", "Filter", "StarID"])
print Data_df[69999:70010].to_latex(columns=["RunningID", "ID", "FieldID", "Filter", "StarID"])

\begin{tabular}{lrrrlr}
\toprule
{} &  RunningID &  ID &  FieldID & Filter &  StarID \\
\midrule
69999 &      12951 &   7 &        1 &      Y &    9999 \\
70000 &     225232 &   8 &        2 &      Z &  170000 \\
70001 &     225233 &   8 &        2 &      Z &  170001 \\
70002 &     225328 &   8 &        2 &      Z &  170002 \\
70003 &     225393 &   8 &        2 &      Z &  170003 \\
70004 &     225330 &   8 &        2 &      Z &  170004 \\
70005 &     225295 &   8 &        2 &      Z &  170005 \\
70006 &     225307 &   8 &        2 &      Z &  170006 \\
70007 &     225308 &   8 &        2 &      Z &  170007 \\
70008 &     225309 &   8 &        2 &      Z &  170008 \\
70009 &     225367 &   8 &        2 &      Z &  170009 \\
\bottomrule
\end{tabular}



In [13]:
print Color_df[:3].to_latex()

\begin{tabular}{lrrrrrrrrr}
\toprule
{} &   colorJH &   colorYJ &  dcolorJH &  dcolorYJ &  StarID &  FieldID &  ClassH &  ClassJ &  ClassY \\
\midrule
0 &  2.450781 & -0.697231 &  0.009473 &  0.003231 &       0 &        1 &      -1 &      -2 &       1 \\
1 &  2.190912 & -0.245317 &  0.006166 &  0.000962 &       1 &        1 &      -1 &       1 &       1 \\
2 &  1.946388 & -0.305107 &  0.005997 &  0.001234 &       2 &        1 &      -1 &      -1 &       1 \\
\bottomrule
\end{tabular}

